In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 

In [2]:
import pandas as pd
import numpy as np
import random as rd
import copy
import scipy.sparse as sp_sparse
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.


In [3]:
# hyperparameters
iterations=100000
alpha=0.01
lambd=0.0000001
maxBreak=10
minImprovement=0.8
minibatch=1000
dropout_ratio=1.0
threads=2

In [4]:
print("------------------Loading data--------------------------")

#Load data
inputDATA="/home/data/t0202002/SLENN/SLE/GeneData.csv"
ClassLabelDATA="/home/data/t0202002/SLENN/SLE/ClassLabels.csv"
# EdgeDATA="/home/data/t0202002/SLENN/SLE/Edgelist.csv"
EdgeDATA="/home/data/t0202002/SLENN/SLE/Edgelist.csv"
outputPath="/home/data/t0202002/SLENN/SLE/results/"


------------------Loading data--------------------------


In [5]:
if not os.path.exists(outputPath):
    os.mkdir(outputPath)
i = 1
while os.path.exists(os.path.join(outputPath, "Run_" + str(i))):
    i += 1
outputPath = os.path.join(outputPath, "Run_" + str(i))
os.mkdir(outputPath)

In [6]:
# gene expression DATA --------------------------------------------------------------------------------------------------------------------
df=pd.read_csv(inputDATA, sep=",", index_col=0)
genelist_sc = df.index.tolist()
cell_sc = df.columns.tolist()
alldata_matrix = sp_sparse.csc_matrix(df.values.astype("float64"))
genelist_sc = [x + "_gene" for x in genelist_sc]

In [7]:
print(alldata_matrix[1:5,1:10])

In [8]:
# classLabelDATA 
def index_match(list2, list1):
    df={}
    result=[]
    for idx, x in enumerate(list1):
        df[x]=idx
    for idx, x in enumerate(list2):
        result.append(df[x])
    return(result)
    
label_data=pd.read_csv(ClassLabelDATA, sep=",")
cell_y = label_data["barcode"].tolist()
barcodes = sorted(set(cell_y) & set(cell_sc))
label_data=label_data.loc[index_match(barcodes, cell_y)]

In [11]:
# Normalization
for i in range(alldata_matrix.shape[1]):
    x=alldata_matrix.data[alldata_matrix.indptr[i]:alldata_matrix.indptr[i+1]]
    alldata_matrix.data[alldata_matrix.indptr[i]:alldata_matrix.indptr[i+1]]=x/x.sum()*1e6
alldata_matrix = alldata_matrix.log1p() 
alldata_matrix = alldata_matrix[index_match(genesList, genelist_sc),:]

In [14]:
# Generate train and test data
test_idx = []
val_idx = []
train_idx = []

if len(test_idx) + len(val_idx) + len(train_idx) != len(barcodes):
    nrTestCells = int(len(barcodes)*0.2) #20% for test
    test_idx = []
    val_idx = []
    train_idx = []
    maxCells=999999
    if nrTestCells > maxCells: nrTestCells = maxCells    
    test_groups = ["".join([str(i) for i in x]) for x in np.transpose(full_data.astype("int")).tolist()]
    test_groups_list={}
    for test_grp in set(test_groups):
        test_groups_list[test_grp] = []
    for idx, x in enumerate(test_groups):
        if not idx in test_idx: 
            test_groups_list[x].append(idx)
    for test_grp in sorted(set(test_groups)):
        test_and_val_group_N = int(((len(test_groups_list[test_grp]) + 0.0)/len(test_groups)) * nrTestCells * 2)
        test_and_val_barcodes=np.random.choice(test_groups_list[test_grp], test_and_val_group_N, replace=False).tolist()
        val_idx_x = np.random.choice(test_and_val_barcodes, int(test_and_val_group_N/2), replace=False).tolist()
        test_idx_x = sorted(set(test_and_val_barcodes) - set(val_idx_x)) 
        train_idx_x = sorted(set(test_groups_list[test_grp]) - set(val_idx_x) - set(test_idx_x))
        print(test_grp + ": test: " + str(len(test_idx_x)) + " val: " + str(len(val_idx_x)) + " train: " + str(len(train_idx_x)) + " of:  " + str(len(test_groups_list[test_grp])))
        test_idx = test_idx + test_idx_x
        val_idx = val_idx + val_idx_x
        train_idx = train_idx + train_idx_x
y_train = full_data[:,train_idx]
valdata_y = full_data[:,val_idx]
testdata_y = full_data[:,test_idx]
x_train = alldata_matrix[:,train_idx]
valdata_x = alldata_matrix[:,val_idx]
testdata_x = alldata_matrix[:,test_idx]

x_train = x_train.tocsr()
valdata_x = valdata_x.tocsr()
testdata_x = testdata_x.tocsr()

0: test: 200 val: 200 train: 600 of:  1000
1: test: 200 val: 200 train: 600 of:  1000


In [16]:
for i in range(alldata_matrix.shape[0]):
    x_train_row=x_train.data[x_train.indptr[i]:x_train.indptr[i+1]]
    valdata_x_row=valdata_x.data[valdata_x.indptr[i]:valdata_x.indptr[i+1]]
    testdata_x_row=testdata_x.data[testdata_x.indptr[i]:testdata_x.indptr[i+1]]
    if x_train_row.shape[0] == x_train.shape[1]:
        x_train_row_min=x_train_row.min()
        x_train_row=x_train_row-x_train_row_min
        valdata_x_row=valdata_x_row-x_train_row_min
        testdata_x_row=testdata_x_row-x_train_row_min
    
    if x_train_row.shape[0] > 0:    
        x_train_row_max = x_train_row.max()
        x_train_row=x_train_row/x_train_row_max
        valdata_x_row=valdata_x_row/x_train_row_max
        testdata_x_row=testdata_x_row/x_train_row_max
    else:
        if valdata_x_row.shape[0] > 0:   
            if valdata_x_row.shape[0] == valdata_x.shape[1]: 
                valdata_x_row=valdata_x_row-valdata_x_row.min()
            valdata_x_row=valdata_x_row/valdata_x_row.max()
        if testdata_x_row.shape[0] > 0:   
            if testdata_x_row.shape[0] == testdata_x.shape[1]: 
                testdata_x_row=testdata_x_row-testdata_x_row.min()
            testdata_x_row=testdata_x_row/testdata_x_row.max()
    
    x_train.data[x_train.indptr[i]:x_train.indptr[i+1]]=x_train_row
    valdata_x.data[valdata_x.indptr[i]:valdata_x.indptr[i+1]]=valdata_x_row
    testdata_x.data[testdata_x.indptr[i]:testdata_x.indptr[i+1]]=testdata_x_row

In [17]:
# cross-entropy loss of each training sample was weighted by the following equation:
def weight_eq(ys_input):
    matrix_groups = ["".join([str(i) for i in x]) for x in np.transpose(ys_input.astype("int")).tolist()]
    matrix_groups_weight={}
    beta=0.01
    for matrix_grp in sorted(set(matrix_groups)):
        matrix_groups_weight[matrix_grp] = (1.0/((matrix_groups.count(matrix_grp) + beta) / len(matrix_groups))) / len(set(matrix_groups))
        #print(matrix_groups_weight[matrix_grp])
    #print(matrix_groups_weight)    
    return(np.array([[matrix_groups_weight[x] for x in matrix_groups],]* ys_input.shape[0]))

In [18]:
# testing data 
valdata_x = valdata_x.toarray() 
valdata_y_weights = weight_eq(valdata_y)
testdata_x = testdata_x.toarray()
testdata_y_weights = weight_eq(testdata_y)

# Shuffle data 
rd.shuffle(genesList)

if minibatch == 0 or minibatch > x_train.shape[1]:
    minibatch = x_train.shape[1]
else:
    minibatch = int(x_train.shape[1]//(x_train.shape[1]//(minibatch)))

In [20]:
# get edge list
edges={}
for index, row in pathway_data.iterrows():
    if(row['parent'] in edges.keys()):
        edges[row['parent']] = edges[row['parent']] + [row['child']]
    else:
        edges[row['parent']] = [row['child']]

In [21]:
GeneMap = {}   
NodeMap = {}   
weightMap = {}    
ids = 0
for iNode in nodesRanks:
    GeneMap[iNode] = []
    NodeMap[iNode] = []
    weightMap[iNode]=[]
    for egde in edges[iNode]:
        weightMap[iNode].append(ids)
        ids += 1
        if egde in genesList:
            GeneMap[iNode].append(genesList.index(egde))
        elif egde in nodesRanks:
            NodeMap[iNode].append(nodesRanks.index(egde))
        else:
            print(egde,"was not found")

weightTotalLength = sum([len(weightMap[i]) for i in nodesRanks])

In [24]:
# Dropout ratio 
dropoutKP_GENES = tf.cast(tf.compat.v1.placeholder_with_default(input=1.0, shape=[]), tf.float64)

output_true = tf.compat.v1.placeholder(shape=[1,None],dtype=tf.float64)
output_weights = tf.compat.v1.placeholder(shape=[1,None],dtype=tf.float64)

# Genes 
genesOrig = tf.compat.v1.placeholder(shape=[x_train.shape[0], None],dtype=tf.float64)
if dropout_ratio < 1:
    genes = tf.nn.dropout(x=genesOrig, rate=1 - (dropoutKP_GENES))
else:
    genes = genesOrig

In [25]:
numApproxPlus = tf.compat.v1.placeholder_with_default(input="", shape=[])
numApproxMinus = tf.compat.v1.placeholder_with_default(input="", shape=[])
numGradEpsilon=0.001
numApproxVec = tf.reshape(tf.cast(tf.tile([numGradEpsilon], [tf.shape(input=genesOrig)[1]],), tf.float64), [1, tf.shape(input=genesOrig)[1]])

# Edge weights
weights = tf.Variable(tf.random.normal([weightTotalLength,1], dtype=tf.float64), dtype=tf.float64)
tf.compat.v1.summary.scalar("mean", tf.reduce_mean(input_tensor=weights))
tf.compat.v1.summary.histogram("histogram", weights)


interceptWeights = tf.Variable(tf.random.normal([len(nodesRanks)], dtype=tf.float64), dtype=tf.float64)
tf.compat.v1.summary.scalar("mean", tf.reduce_mean(input_tensor=interceptWeights))
tf.compat.v1.summary.histogram("histogram", interceptWeights)

Instructions for updating:
Colocations handled automatically by placer.


<tf.Tensor 'histogram_1:0' shape=() dtype=string>

idxs = list(range(x_train.shape[1]))
np.random.shuffle(idxs)
minibatch_list = [idxs[j:min(j+minibatch, len(idxs)-1)] for j in list(range(0, x_train.shape[1], minibatch))]
minibatch_list_length = [len(x) for x in minibatch_list]
print(len(idxs),"idxs")

for i_batch in range(len(minibatch_list)):
    print(i_batch,"i_batch")
    batch_idx=minibatch_list[i_batch]
    x_batch=x_train[:,batch_idx].toarray()
    with tf.Session() as sess:
        a=sess.run(genesOrig,{genesOrig:x_batch})
        print(a,a.shape)
        b=sess.run(numApproxVec,{genesOrig:x_batch})
        print(b,b.shape)

In [ ]:
# Dropout nodes
nodes = {}
nodes_loss = {}
genes_unstacked = tf.unstack(genes)
for n in nodesRanks:
    weightsX = tf.slice(weights, [weightMap[n][0],0],[len(weightMap[n]),1])
    features = tf.stack([genes_unstacked[x] for x in GeneMap[n]] + [nodes[nodesRanks[nidx]] for nidx in NodeMap[n]])
    z = tf.matmul(tf.transpose(a=weightsX), features) 
    z = z + tf.slice(interceptWeights, [nodesRanks.index(n)],[1])
    if n=="output":
        nodes[n] = z
    else:
        nodes[n] = tf.nn.sigmoid(z)        

    nodes[n] = tf.cond(pred=tf.equal(numApproxPlus, tf.constant(n)), true_fn=lambda: nodes[n] + numApproxVec, false_fn=lambda: nodes[n])
    nodes[n] = tf.cond(pred=tf.equal(numApproxMinus, tf.constant(n)), true_fn=lambda: nodes[n] - numApproxVec, false_fn=lambda: nodes[n])

    nodes[n] = tf.unstack(nodes[n])[0]
    # print(n, nodes[n])
    
    
    if not n in outputs:
        parents = pathway_data[pathway_data['child'].str.match("^" + n + "$")]['parent'].tolist()
        children = len(set(pathway_data[pathway_data['parent'].isin(parents)]["child"].tolist()))
        #print(parents,children)
        if children == 1:
            print("Skipping dropout for " + n)
        else:
            nodes[n] = tf.nn.dropout(x=nodes[n], rate=1 - dropout_ratio)
            # print(nodes[n])

In [28]:
# Evaluation results
weightsToPenalize = weights
regularization = (lambd * tf.nn.l2_loss(weightsToPenalize)) / tf.cast(tf.shape(input=output_true)[1], tf.float64)
# tf.compat.v1.summary.scalar("value", regularization)

xentropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=tf.stack([nodes[x] for x in outputs]), labels=output_true) * output_weights
# tf.compat.v1.summary.scalar("mean", tf.reduce_mean(input_tensor=xentropy))
# tf.compat.v1.summary.histogram("histogram", xentropy)

loss = tf.reduce_mean(input_tensor=xentropy) + regularization
# tf.compat.v1.summary.scalar("value", loss)

y_hat = tf.nn.sigmoid(tf.stack([nodes[x] for x in outputs]))

error = tf.abs(output_true - y_hat) * output_weights
# tf.compat.v1.summary.scalar("mean", tf.reduce_mean(input_tensor=error))

accuracy = tf.reduce_mean(input_tensor=tf.cast(tf.equal(tf.round(y_hat), output_true), tf.float64))
# tf.compat.v1.summary.scalar("mean", tf.reduce_mean(input_tensor=accuracy))

In [29]:
# AdamOptimizer
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=alpha)    
train = optimizer.minimize(loss)

In [30]:
# model Initialization
print("----------------\n...Initializing")
init = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(inter_op_parallelism_threads=threads,intra_op_parallelism_threads=threads))
sess.run(init)


merged = tf.compat.v1.summary.merge_all()
saver = tf.compat.v1.train.Saver()
sep=","

costFile = open(os.path.join(outputPath, "performance.csv"),"w")
costFile.write("iteration,train,validation,error,accuracy,BreakCount" + "\n")
costFile.close()

----------------
...Initializing


idxs = list(range(x_train.shape[1]))
np.random.shuffle(idxs)
minibatch_list = [idxs[j:min(j+minibatch, len(idxs)-1)] for j in list(range(0, x_train.shape[1], minibatch))]
minibatch_list_length = [len(x) for x in minibatch_list]
print(len(idxs),"idxs")

with tf.Session() as sess:
    for i_batch in range(len(minibatch_list)):
        print(i_batch,"i_batch")
        batch_idx=minibatch_list[i_batch]
        x_batch=x_train[:,batch_idx].toarray()
        y_batch=y_train[:,batch_idx]
        y_batch_weights = weight_eq(y_batch)
        lossRes, trainClassProb, trainWriteContent = sess.run([loss, y_hat, merged], {genesOrig:x_batch, output_true:y_batch, dropoutKP_GENES: dropout_ratio, output_weights:y_batch_weights})


batch_idx=minibatch_list[0]
x_batch=x_train[:,batch_idx]
y_batch=y_train[:,batch_idx]
print(type(y_batch),"什么")
y_batch=y_train[:,batch_idx]
y_batch_weights = weight_eq(y_batch)
x_batch,y_batch,y_batch_weights,x_batch.shape,y_batch.shape,y_batch_weights.shape

In [31]:
testdata_x.shape

(3504, 400)

In [ ]:
# Model training
trainCost = []
valErr = []
valErrCurrentMin = 1
BreakCount = 0

for i in [xx + 1 for xx in range(iterations)]:    
    # DEFINE MINIBATCH    
    idxs = list(range(x_train.shape[1]))
    np.random.shuffle(idxs)
    minibatch_list = [idxs[j:min(j+minibatch, len(idxs)-1)] for j in list(range(0, x_train.shape[1], minibatch))]
    minibatch_list_length = [len(x) for x in minibatch_list]
    if 0 in minibatch_list_length:
        minibatch_list_idx_remove = minibatch_list_length.index(0)
        del minibatch_list[minibatch_list_idx_remove]
    for i_batch in range(len(minibatch_list)):
        batch_idx=minibatch_list[i_batch]
        x_batch=x_train[:,batch_idx].toarray()
        y_batch=y_train[:,batch_idx]
        y_batch_weights = weight_eq(y_batch)
       dropout_ratio=dropout_ratio*N0/(N0+1)*(dropout_ratio** 0.5)
        if i > 1: 
            sess.run(train, {genesOrig:x_batch, output_true:y_batch, dropoutKP_GENES:dropout_ratio, output_weights:y_batch_weights})

    if i < 10 or i % 10 == 0:
        print("\nTraining epoch: " + str(i))
        # lossRes=sess.run(loss, {genesOrig:x_batch, output_true:y_batch, dropoutKP_GENES: dropout_ratio, output_weights:y_batch_weights})
        lossRes, trainClassProb, trainWriteContent = sess.run([loss, y_hat, merged], {genesOrig:x_batch, output_true:y_batch, dropoutKP_GENES: dropout_ratio, output_weights:y_batch_weights})
        lossValRes, valWriteContent, valerror, valAccuracy = sess.run([loss, merged, error,accuracy], {genesOrig:valdata_x, output_true:valdata_y, output_weights:valdata_y_weights})
        valErrRun = valerror.mean()
                
        # Early stopping
        if i > 10:
            if trainCost[-1] - lossRes < 0.00001: 
                BreakCount += 1
            if any(valErrPrevious + 0.05 < valErrRun for valErrPrevious in valErr): 
                BreakCount += 1
      
        # record the training progress
        print("Mean loss: " + "%.4f" % lossRes + " Validation loss: " + "%.4f" % lossValRes + " Validation error: " + str(valErrRun))
        print("Break Counter = " + str(BreakCount))
        costFile = open(os.path.join(outputPath, "performance.csv"),"a")
        costFile.write(str(i) + sep + str(lossRes) + sep + str(lossValRes) + sep + str(valErrRun) + sep + str(valAccuracy.mean()) + sep + str(BreakCount) + sep + str(dropout_ratio) + "\n")
        costFile.close()
        
        if BreakCount > maxBreak:
            break
        
        # Save model
        if i > 2 and (valErrRun < 0.2 and valErrCurrentMin * minImprovement > valErrRun):
            BreakCount = 0
            saver.save(sess, os.path.join(outputPath,'Best-model'))
            valErrCurrentMin = valErrRun
            print("Model saved at iteration " + str(i) + " at Validation error: " + str(valErrRun))
        saver.save(sess, os.path.join(outputPath,'Best-model'))
        trainCost.append(lossRes)
        valErr.append(valErrRun)
        
# Load the best model
if os.path.exists(os.path.join(outputPath,'Best-model.meta')):
    saver.restore(sess, os.path.join(outputPath,'Best-model'))
    print("Model restored from " + outputPath)

ValidationError = sess.run(error, {genesOrig:testdata_x, output_true:testdata_y, output_weights:testdata_y_weights}).mean()
print("ValidationError:" + str(ValidationError))
open(os.path.join(outputPath, "ValidationError.txt"),"w").write("ValidationError" + "\n" + str(ValidationError))

# record predict results
output_predict = sess.run(y_hat, {genesOrig:testdata_x, output_true:testdata_y})
output_predict_file = open(os.path.join(outputPath, "output_predict.csv"),"w")
output_predict_file.write(",".join(outputs) + "\n")
output_predict_file.write(pd.DataFrame(np.transpose(output_predict)).to_csv(header=False, index=False))
output_predict_file.close()
# real data
output_real_file = open(os.path.join(outputPath, "output_real.csv"),"w")
output_real_file.write(",".join(outputs) + "\n")
output_real_file.write(pd.DataFrame(np.transpose(testdata_y.astype("int"))).to_csv(header=False, index=False))
output_real_file.close()

# Weight output
sep = ","
weightRes, interceptRes = sess.run([weights, interceptWeights])
weightFile = open(os.path.join(outputPath, "Edge_weights.csv"),"w")
weightFile.write("parent,child,weight" + "\n")
for n in nodesRanks:
    i = 0
    for targetGene in GeneMap[n]:
        weightFile.write(n + sep + genesList[targetGene] + sep + str(weightRes[weightMap[n][i]].tolist()[0]) + "\n")
        i += 1
    for targetProtein in NodeMap[n]:
        weightFile.write(n + sep + nodesRanks[targetProtein] + sep + str(weightRes[weightMap[n][i]].tolist()[0]) + "\n")
        i += 1
    assert i == len(weightMap[n])
    weightFile.write(n + sep + "Intercept" + sep + str(interceptRes[nodesRanks.index(n)]) + "\n")

weightFile.close()


print("Calculate contribution scores-------------------------------")

# calculate contribution scores 
numAgg = 0
nodesNumGrad = [x for x in nodesRanks if not x in outputs]
for n in nodesNumGrad:
    res = (sess.run(y_hat, {genesOrig:testdata_x, output_true:testdata_y, numApproxPlus: n}) - sess.run(y_hat, {genesOrig:testdata_x, output_true:testdata_y, numApproxMinus: n}))/(2*numGradEpsilon)
    res = res.mean(1)
    if numAgg.__class__ != np.ndarray:
        numAgg = res
    else:
        numAgg = np.column_stack((numAgg, res))

# save contribution scores results
sep = ","
Contri_score = open(os.path.join(outputPath, "Contri_score.csv"),"w")
Contri_score.write("Gene" + sep + sep.join(outputs) + "\n")
for i,n in enumerate(nodesNumGrad):
    Contri_score.write(n + sep + sep.join([str(x) for x in numAgg[:,i].tolist()]) + "\n")

Contri_score.close()

sess.close()

print("Model was successfully completed-------------------------------")
print("\nResults are exported to " + outputPath +"\n\n")